In [43]:
!pip install pdfplumber


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [44]:
import pdfplumber
import json

def pdf_to_json(pdf_path):
    # Initialize an empty dictionary to store the page content
    pdf_data = {}

    # Open the PDF file
    with pdfplumber.open(pdf_path) as pdf:
        # Iterate through each page
        for i, page in enumerate(pdf.pages):
            # Extract text from the page
            page_text = page.extract_text()

            # Store the extracted text in the dictionary with the page number as the key
            pdf_data[f"Page_{i+1}"] = page_text.strip() if page_text else ""

    # Convert the dictionary to a JSON string
    pdf_json = json.dumps(pdf_data, indent=4)

    return pdf_json

def save_json_to_file(json_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)

if __name__ == "__main__":
    pdf_path = "BH24_Hacking_Groups.pdf"  # Replace with your PDF file path
    output_path = "output.json"  # Replace with desired output file path

    # Convert PDF to JSON
    pdf_json = pdf_to_json(pdf_path)

    # Save JSON to file
    save_json_to_file(pdf_json, output_path)

    print(f"PDF has been converted to JSON and saved to {output_path}")


PDF has been converted to JSON and saved to output.json


In [45]:
# prompt: load the JSON file to an array of dictionaries

import json

def load_json(file_path):
  with open(file_path, 'r') as f:
    data = json.load(f)
  return [{"page": key, "content": value} for key, value in data.items()]

# Example usage:
json_array = load_json("output.json")
print(json_array)


[{'page': 'Page_1', 'content': 'Hacking groups\nBioHackathon 2024'}, {'page': 'Page_2', 'content': 'Use this slide as a template\nGroup topic\nParticipants\n← Put your names here to participate in\n● Toshiaki, …\nUse the bold font for group lead\nDescription\n● ← Describe objectives, requirements etc.\n(Group leader is responsible for this\nand presentation)\nWe will start presentations and reviews\nof the hacking projects at 11:00'}, {'page': 'Page_3', 'content': 'Suggested target domains\n● [R1] Multi-omics analysis on human genotype to phenotype that includes genomic, transcriptomic, epigenomic,\nproteomic, protein structures, and biochemical data.\n● [R2] Automated data analysis of other organisms including phylogenetic compositions, gene annotations, pathways,\nand growth conditions.\n● [R3] Data-driven interdisciplinary studies in public health, environment, agriculture, food, energy, and other fields\nutilizing knowledge graphs.\n● [R4] Facilitating knowledge discovery and biolo

In [46]:
# prompt: remove pages 1-4

# Filter out pages 1-4
filtered_json_array = [item for item in json_array if int(item["page"].split("_")[1]) > 4]

print(filtered_json_array)


[{'page': 'Page_5', 'content': 'Genome variation\nParticipants\n● Yosuke, Toshiaki, Maxat, Toyoyuki, Yuki, Nobutaka, Hirokazu, Tsuyoshi, Dorothy, Pitiporn (Sam),\nMayumi, Núria, Shuichi, Kentaro (Yamaken), David, Takatomo, Hiroyuki Mishima, Tazro\nMotivation\nThe analysis of the human genome has been flooded with data due to the widespread use of\nsequencers. The simple variations such as SNV and indel are being integrated with TogoVar, but\nstructural variation has not yet even been standardized. Meanwhile, the pangenome graphs has\nemerged as a powerful tool for integrating multiple haplotypes. During this hackathon, we would\nlike to discuss how to handle this heterogeneous data in a unified manner.\nDescription (Write down your proposal here.)\n● Genome graph - new repository -> move to new project\n● SV, CNV, STR, …\n● Clinical variants - MGeND\n● https://www.ga4gh.org/product/variatioepresentation/ (are you also interested?👍)\n● Workflow（Tool Pipeline）n-r\n● Genome Variation Onto

In [47]:
# prompt: modify the array to add a title for each item where the first line of the content is the title

def add_titles(json_array):
  for item in json_array:
    content_lines = item["content"].splitlines()
    if content_lines:
      item["title"] = content_lines[0]
    else:
      item["title"] = ""
  return json_array

# Apply the function to the filtered JSON array
titled_json_array = add_titles(filtered_json_array)
titled_json_array = [item for item in titled_json_array if item['title'] not in [
    'R4', 'R3', 'R2', 'R1', 'Group photo', 'Self-Introduction', 'Additionals', 'Request for tutorials']]

print(titled_json_array)


[{'page': 'Page_5', 'content': 'Genome variation\nParticipants\n● Yosuke, Toshiaki, Maxat, Toyoyuki, Yuki, Nobutaka, Hirokazu, Tsuyoshi, Dorothy, Pitiporn (Sam),\nMayumi, Núria, Shuichi, Kentaro (Yamaken), David, Takatomo, Hiroyuki Mishima, Tazro\nMotivation\nThe analysis of the human genome has been flooded with data due to the widespread use of\nsequencers. The simple variations such as SNV and indel are being integrated with TogoVar, but\nstructural variation has not yet even been standardized. Meanwhile, the pangenome graphs has\nemerged as a powerful tool for integrating multiple haplotypes. During this hackathon, we would\nlike to discuss how to handle this heterogeneous data in a unified manner.\nDescription (Write down your proposal here.)\n● Genome graph - new repository -> move to new project\n● SV, CNV, STR, …\n● Clinical variants - MGeND\n● https://www.ga4gh.org/product/variatioepresentation/ (are you also interested?👍)\n● Workflow（Tool Pipeline）n-r\n● Genome Variation Onto

In [6]:
# prompt: go through the titles and remove the text "\u21d4\n" from them if they are there

def clean_titles(json_array):
  for item in json_array:
    item["title"] = item["title"].replace("\u21d4\n", "")
  return json_array

titled_json_array = clean_titles(titled_json_array)
print(json_array)


[{'page': 'Page_1', 'content': 'Hacking groups\nBioHackathon 2024'}, {'page': 'Page_2', 'content': 'Use this slide as a template\nGroup topic\nParticipants\n← Put your names here to participate in\n● Toshiaki, …\nUse the bold font for group lead\nDescription\n● ← Describe objectives, requirements etc.\n(Group leader is responsible for this\nand presentation)\nWe will start presentations and reviews\nof the hacking projects at 11:00'}, {'page': 'Page_3', 'content': 'Suggested target domains\n● [R1] Multi-omics analysis on human genotype to phenotype that includes genomic, transcriptomic, epigenomic,\nproteomic, protein structures, and biochemical data.\n● [R2] Automated data analysis of other organisms including phylogenetic compositions, gene annotations, pathways,\nand growth conditions.\n● [R3] Data-driven interdisciplinary studies in public health, environment, agriculture, food, energy, and other fields\nutilizing knowledge graphs.\n● [R4] Facilitating knowledge discovery and biolo

In [7]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

embedded = [model.encode(item.get('content', '')) for item in titled_json_array]


  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.1 MB/s eta 0:00:00
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.9 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sentence_transfor

/Users/davidcs/git/dbcls-umap/env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/davidcs/git/dbcls-umap/env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
!pip install umap-learn
!pip install numpy==2.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 5.8 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import umap
reducer = umap.UMAP(n_neighbors=5, min_dist=0.1, n_components=8)

ImportError: Numba needs NumPy 2.0 or less. Got NumPy 2.1.

In [13]:
umapped = reducer.fit_transform(embedded)

NameError: name 'reducer' is not defined

In [21]:
!pip list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Package                   Version
------------------------- --------------
anyio                     4.4.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.7.4
cffi                      1.17.0
charset-normalizer        3.3.2
comm                      0.2.2
cryptography              43.0.0
debugpy                   1.8.5
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.1
fastjsonschema            2.20.0
filelock                  3.15.4
fqdn                      1.5.1
fsspec                    2024.6.1
h11                       0.14.0
httpcore                  1.0.5
httpx                     0.27.0
huggingface-hub           0.24.6
idna       

In [17]:
!pip list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Package                   Version
------------------------- --------------
anyio                     4.4.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.7.4
cffi                      1.17.0
charset-normalizer        3.3.2
comm                      0.2.2
cryptography              43.0.0
debugpy                   1.8.5
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.0.1
fastjsonschema            2.20.0
filelock                  3.15.4
fqdn                      1.5.1
fsspec                    2024.6.1
h11                       0.14.0
httpcore                  1.0.5
httpx                     0.27.0
huggingface-hub           0.24.6
idna       

In [ ]:
!pip install plotly

In [ ]:

import plotly.express as px

fig = px.scatter(x=umapped[:,0], y=umapped[:,1],
                 text=[item['title'] for item in titled_json_array],
                 height=800)
fig.update_traces(textposition='top center')
fig.show()


In [14]:
!pip install huggingface_hub
import huggingface_hub
huggingface_hub.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 63.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [18]:
import transformers
import torch
import huggingface_hub

model_id = "meta-llama/Meta-Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, max_new_tokens=500
)



model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [19]:
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 6.2 MB/s eta 0:00:00a 0:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 12.9 MB/s eta 0:00:00
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.8 MB/s eta 0:00:00a 0:00:01
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [48]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="1234",
    base_url="http://localhost:1234/v1/",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf",
)

print(chat_completion)

ChatCompletion(id='chatcmpl-4hpvbzqssir6a7k0ork8k9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.\nThe response follows the exact words given in the first sentence and is therefore the best completion to the task.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1724728394, model='lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=30, total_tokens=56))


In [49]:
def pipeline(prompt):
    return client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_tokens=500,
        model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q3_K_L.gguf",
    ).choices[0].message.content

In [52]:
# prompt: iterate through each of the items of the titled_json_array and run pipeline on the content field and add it to the array under a new key

print(len(titled_json_array))
i = 0
for item in titled_json_array:
  print(i)
  item['generated_text'] = pipeline(f"Write an informed and brief abstract based on the content of this slide that would be appropriate for reporting results to the wider bioinformatics community. Do not include any conversational response, just the abstract {item['content']}")
  i += 1

31
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [50]:
titled_json_array

[{'page': 'Page_5',
  'content': 'Genome variation\nParticipants\n● Yosuke, Toshiaki, Maxat, Toyoyuki, Yuki, Nobutaka, Hirokazu, Tsuyoshi, Dorothy, Pitiporn (Sam),\nMayumi, Núria, Shuichi, Kentaro (Yamaken), David, Takatomo, Hiroyuki Mishima, Tazro\nMotivation\nThe analysis of the human genome has been flooded with data due to the widespread use of\nsequencers. The simple variations such as SNV and indel are being integrated with TogoVar, but\nstructural variation has not yet even been standardized. Meanwhile, the pangenome graphs has\nemerged as a powerful tool for integrating multiple haplotypes. During this hackathon, we would\nlike to discuss how to handle this heterogeneous data in a unified manner.\nDescription (Write down your proposal here.)\n● Genome graph - new repository -> move to new project\n● SV, CNV, STR, …\n● Clinical variants - MGeND\n● https://www.ga4gh.org/product/variatioepresentation/ (are you also interested?👍)\n● Workflow（Tool Pipeline）n-r\n● Genome Variation On

In [54]:
# prompt: iterate through each of the entries in the array and add them to a markdown file with the title as the heading and the paragraph between them the generated text

with open('output.md', 'w') as f:
  for item in titled_json_array:
    f.write(f"# {item['title']}\n\n")
    f.write(item['generated_text'] + "\n\n")


In [1]:
titled_json_array

NameError: name 'titled_json_array' is not defined